In [226]:
from ast import literal_eval
import re

import numpy as np
import pandas as pd

##Set the Columns to show all the data
pd.set_option("display.max_columns", 100)

In [227]:
# read in the previously downloaded data
raw_steam_data = pd.read_csv("downloads/steam_app_data.csv")

# print out number of rows and columns
print("Rows:", raw_steam_data.shape[0])
print("Columns:", raw_steam_data.shape[1])

# view first five rows
raw_steam_data.head()

Rows: 50
Columns: 39


,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
0,game,Counter-Strike,10,0,False,NaN,NaN,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,NaN,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'EUR', 'initial': 819, 'final': 8...","[574941, 7]","[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","{'score': 88, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 148903},NaN,"{'coming_soon': False, 'date': '1 Nov, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
1,game,Team Fortress Classic,20,0,False,NaN,NaN,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,NaN,"English, French, German, Italian, Spanish - Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'GBP', 'initial': 429, 'final': 4...",[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}",NaN,NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 6000},NaN,"{'coming_soon': False, 'date': '1 Apr, 1999'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
2,game,Day of Defeat,30,0,False,NaN,NaN,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,NaN,"English, French, German, Italian, Spanish - Spain",https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'GBP', 'initial': 429, 'final': 4...","[30, 944613]","[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","{'score': 79, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 4008},NaN,"{'coming_soon': False, 'date': '1 May, 2003'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
3,game,Deathmatch Classic,40,0,False,NaN,NaN,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,NaN,"English, French, German, Italian, Spanish - Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum

In [228]:
##Find the amount of null data in the csv
null_counts = raw_steam_data.isnull().sum()
null_counts

type                        0
name                        0
steam_appid                 0
required_age                0
is_free                     0
controller_support         40
dlc                        38
detailed_description        0
about_the_game              0
short_description           0
fullgame                   50
supported_languages         0
header_image                0
website                    15
pc_requirements             0
mac_requirements            0
linux_requirements          0
legal_notice               36
drm_notice                 50
ext_user_account_notice    50
developers                  0
publishers                  0
demos                      45
price_overview              7
packages                    4
package_groups              0
platforms                   0
metacritic                 13
reviews                    44
categories                  0
genres                      0
screenshots                 0
movies                     27
recommenda

In [229]:
##Drop columns with more than 50% missing values.
threshold = raw_steam_data.shape[0] // 2

print("Drop columns with more than {} missing rows".format(threshold))
print()

drop_rows = raw_steam_data.columns[null_counts > threshold]

print("Columns to drop: {}".format(list(drop_rows)))

Drop columns with more than 25 missing rows

Columns to drop: ['controller_support', 'dlc', 'fullgame', 'legal_notice', 'drm_notice', 'ext_user_account_notice', 'demos', 'reviews', 'movies', 'achievements']


In [230]:
print("Rows to remove:", raw_steam_data[raw_steam_data["type"].isnull()].shape[0])

# Preview rows with no data in column "type"
raw_steam_data[raw_steam_data["type"].isnull()].head()

Rows to remove: 0


,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors


In [231]:
##Drop these column as it holds no value anymore
raw_steam_data["type"].value_counts(dropna = False)
raw_steam_data["screenshots"].value_counts(dropna = False)
raw_steam_data["movies"].value_counts(dropna = False)
raw_steam_data["controller_support"].value_counts(dropna = False)
raw_steam_data["dlc"].value_counts(dropna = False)
raw_steam_data["header_image"].value_counts(dropna = False)
raw_steam_data["legal_notice"].value_counts(dropna = False)
raw_steam_data["drm_notice"].value_counts(dropna = False)
raw_steam_data["ext_user_account_notice"].value_counts(dropna = False)
raw_steam_data["demos"].value_counts(dropna = False)
raw_steam_data["packages"].value_counts(dropna = False)
raw_steam_data["release_date"].value_counts(dropna = False)
raw_steam_data["support_info"].value_counts(dropna = False)
raw_steam_data["background"].value_counts(dropna = False)
raw_steam_data["content_descriptors"].value_counts(dropna = False)
raw_steam_data["achievements"].value_counts(dropna = False)


achievements
NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [232]:
##checks for any games with missing names
raw_steam_data[(raw_steam_data["name"].isnull()) | (raw_steam_data["name"] == "none")]

,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors


In [233]:
##Remove Duplicate rows
duplicate_rows = raw_steam_data[raw_steam_data.duplicated()]

print("Duplicate rows to remove:", duplicate_rows.shape[0])

duplicate_rows.head()

Duplicate rows to remove: 1


,type,name,steam_appid,required_age,is_free,controller_support,dlc,detailed_description,about_the_game,short_description,fullgame,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,legal_notice,drm_notice,ext_user_account_notice,developers,publishers,demos,price_overview,packages,package_groups,platforms,metacritic,reviews,categories,genres,screenshots,movies,recommendations,achievements,release_date,support_info,background,content_descriptors
8,game,Counter-Strike: Condition Zero,80,0,False,NaN,NaN,"With its extensive Tour of Duty campaign, a ne...","With its extensive Tour of Duty campaign, a ne...","With its extensive Tour of Duty campaign, a ne...",NaN,"English, French, German, Italian, Spanish - Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,[],[],NaN,NaN,NaN,['Valve'],['Valve'],NaN,"{'currency': 'GBP', 'initial': 719, 'final': 7...",[7],"[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","{'score': 65, 'url': 'https://www.metacritic.c...",NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",NaN,{'total': 18233},NaN,"{'coming_soon': False, 'date': '1 Mar, 2004'}","{'url': 'http://steamcommunity.com/app/80', 'e...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"


In [234]:
def dropNullCollumns(df, thresh=0.5):
    ##Drop columns with more than 50% of missing values
    cutoff_count = len(df) * thresh
    
    return df.dropna(thresh = cutoff_count, axis = 1)

def processNameType(df):
    ##Remove null values in name and type columns and then remove the type column
    df = df[df["type"].notnull()]
    
    df = df[df["name"].notnull()]
    df = df[df["name"] != "none"]
    
    df = df.drop("type", axis = 1)
    
    return df

def process(df):
    ##Process the data set (Used temporarily to modify the datafram and a final process function is created at the end)
    
    ##Make a Copy of the dataframe so you dont have to redownlaod the data each time i make a mistake
    df = df.copy()
    
    ##Remove duplicate rows
    df = df.drop_duplicates()
    
    ##Remove collumns with more than 50% null values
    df = dropNullCollumns(df)
    
    ##Process rest of columns
    df = processNameType(df)
    
    return df

print(raw_steam_data.shape)
initial_processing = process(raw_steam_data)
print(initial_processing.shape)
initial_processing.head()


(50, 39)
(49, 28)


,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,metacritic,categories,genres,screenshots,recommendations,release_date,support_info,background,content_descriptors
0,Counter-Strike,10,0,False,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,"English<strong>*</strong>, French<strong>*</st...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'EUR', 'initial': 819, 'final': 8...","[574941, 7]","[{'name': 'default', 'title': 'Buy Counter-Str...","{'windows': True, 'mac': True, 'linux': True}","{'score': 88, 'url': 'https://www.metacritic.c...","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 148903},"{'coming_soon': False, 'date': '1 Nov, 2000'}","{'url': 'http://steamcommunity.com/app/10', 'e...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
1,Team Fortress Classic,20,0,False,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,"English, French, German, Italian, Spanish - Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'GBP', 'initial': 429, 'final': 4...",[29],"[{'name': 'default', 'title': 'Buy Team Fortre...","{'windows': True, 'mac': True, 'linux': True}",NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 6000},"{'coming_soon': False, 'date': '1 Apr, 1999'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [2, 5], 'notes': 'Includes intense vio..."
2,Day of Defeat,30,0,False,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,"English, French, German, Italian, Spanish - Spain",https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'GBP', 'initial': 429, 'final': 4...","[30, 944613]","[{'name': 'default', 'title': 'Buy Day of Defe...","{'windows': True, 'mac': True, 'linux': True}","{'score': 79, 'url': 'https://www.metacritic.c...","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 4008},"{'coming_soon': False, 'date': '1 May, 2003'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
3,Deathmatch Classic,40,0,False,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,"English, French, German, Italian, Spanish - Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],"{'currency': 'GBP', 'initial': 429, 'final': 4...",[31],"[{'name': 'default', 'title': 'Buy Deathmatch ...","{'windows': True, 'mac': True, 'linux': True}",Na

In [235]:
##Display the required age count and sort them
initial_processing["required_age"].value_counts(dropna = False).sort_index()

required_age
0     44
16     3
18     2
Name: count, dtype: int64

In [236]:
def processAge(df):
    ##Format ratings in age column to be in line with the PEGI ratings 
    # PEGI Age ratings: 3, 7, 12, 16, 18
    cut_points = [-1, 0, 3, 7, 12, 16, 2000]
    label_values = [0, 3, 7, 12, 16, 18]
    
    df["required_age"] = pd.cut(df["required_age"], bins = cut_points, labels = label_values)
    
    return df


age_df = processAge(initial_processing)
age_df["required_age"].value_counts().sort_index()

required_age
0     44
3      0
7      0
12     0
16     3
18     2
Name: count, dtype: int64

In [237]:
##Preview the price_overview
age_df["price_overview"].head()

0    {'currency': 'EUR', 'initial': 819, 'final': 8...
1    {'currency': 'GBP', 'initial': 429, 'final': 4...
2    {'currency': 'GBP', 'initial': 429, 'final': 4...
3    {'currency': 'GBP', 'initial': 429, 'final': 4...
4    {'currency': 'GBP', 'initial': 429, 'final': 4...
Name: price_overview, dtype: object

In [238]:
##Preview the platforms
age_df["platforms"].head()

0    {'windows': True, 'mac': True, 'linux': True}
1    {'windows': True, 'mac': True, 'linux': True}
2    {'windows': True, 'mac': True, 'linux': True}
3    {'windows': True, 'mac': True, 'linux': True}
4    {'windows': True, 'mac': True, 'linux': True}
Name: platforms, dtype: object

In [239]:
##The data in the rows are stored as a string even though it looks like a dictionary
platforms_first_row = age_df["platforms"].iloc[0]

print(type(platforms_first_row))

platforms_first_row

<class 'str'>


"{'windows': True, 'mac': True, 'linux': True}"

In [240]:
##using the eval function it evaluets teh string and indexes it as a dictionary
eval_first_row = literal_eval(platforms_first_row)

print(type(eval_first_row))
print(eval_first_row)

eval_first_row["windows"]

<class 'dict'>
{'windows': True, 'mac': True, 'linux': True}


True

In [241]:
#Check for any missing values
age_df["platforms"].isnull().sum()

0

In [242]:
##Create string of keys joined on a semi-colon
";".join(eval_first_row.keys())

'windows;mac;linux'

In [243]:
platforms = {"windows": True, "mac": True, "linux": False}

# list comprehension
print([x for x in platforms.keys() if platforms[x]])

# using list comprehension in join
";".join(x for x in platforms.keys() if platforms[x])

['windows', 'mac']


'windows;mac'

In [244]:
def processPlatforms(df):
    ##Split the platforms column into separate columns for each platform

    ##Evaluate values in the platforms column so it can index into dictionaries
    df = df.copy()
    
    def parsePlatforms(x):
        
        d = literal_eval(x)
        
        return ";".join(platform for platform in d.keys() if d[platform])
    
    df["platforms"] = df["platforms"].apply(parsePlatforms)
    
    return df


platforms_df = processPlatforms(age_df)
platforms_df["platforms"].value_counts()

platforms
windows;mac;linux    27
windows              16
windows;mac           5
windows;linux         1
Name: count, dtype: int64

In [245]:
##Checks for any null data in price overvriew
platforms_df["price_overview"].isnull().sum()

7

In [246]:
##Combine the free games with games that have null price
free_and_null_price = platforms_df[(platforms_df["is_free"]) & (platforms_df["price_overview"].isnull())]

free_and_null_price.shape[0]

5

In [247]:
##Combine rows that are not free, but have null price values
not_free_and_null_price = platforms_df[(platforms_df["is_free"] == False) & (platforms_df["price_overview"].isnull())]

not_free_and_null_price.head()

,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,price_overview,packages,package_groups,platforms,metacritic,categories,genres,screenshots,recommendations,release_date,support_info,background,content_descriptors
15,Half-Life 2: Lost Coast,340,0,False,Originally planned as a section of the Highway...,Originally planned as a section of the Highway...,Originally planned as a section of the Highway...,"English, French, German, Italian, Spanish - Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.half-life2.com,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,[],[],['Valve'],['Valve'],NaN,NaN,[],windows;mac;linux,NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 10099},"{'coming_soon': False, 'date': '27 Oct, 2005'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"
44,Rome: Total War™ - Collection,4760,0,False,<h1>Total War: ROME REMASTERED announced!</h1>...,"Once the Roman Empire is under your command, d...",Control and conquer the greatest empire ever k...,English,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.totalwar.com/,{'minimum': '<strong>Minimum: </strong>Microso...,[],[],['CREATIVE ASSEMBLY'],['SEGA'],NaN,NaN,[],windows,"{'score': 92, 'url': 'https://www.metacritic.c...","[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '2', 'description': 'Strategy'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 12112},"{'coming_soon': False, 'date': '28 Aug, 2007'}","{'url': 'https://support.sega.co.uk', 'email':...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}"


In [248]:
def printSteamLinks(df):
    ##Print links to store page for apps
    url_base = "https://store.steampowered.com/app/"
    
    for i, row in df.iterrows():
        appid = row["steam_appid"]
        name = row["name"]
        
        print(name + ":", url_base + str(appid))
        

printSteamLinks(not_free_and_null_price[:5])

Half-Life 2: Lost Coast: https://store.steampowered.com/app/340
Rome: Total War™ - Collection: https://store.steampowered.com/app/4760


In [249]:
##Preview the price for the first column
platforms_df["price_overview"][0]

"{'currency': 'EUR', 'initial': 819, 'final': 819, 'discount_percent': 0, 'initial_formatted': '', 'final_formatted': '8,19€'}"

In [250]:
def processPrice(df):
    df = df.copy()
        
    def parsePrice(x):
        if x is not np.nan:
            return literal_eval(x)
        else:
            return {"currency": "GBP", "initial": -1}
    
    ##Evaluate as dictionary and set to -1 if price is missing
    df["price_overview"] = df["price_overview"].apply(parsePrice)
    
    ##Create columns from currency and initial values
    df["currency"] = df["price_overview"].apply(lambda x: x["currency"])
    df["price"] = df["price_overview"].apply(lambda x: x["initial"])
    
    ##Set price of free games to 0
    df.loc[df["is_free"], "price"] = 0
    
    return df

price_data = processPrice(platforms_df)[["name", "currency", "price"]]
price_data.head()

,name,currency,price
0,Counter-Strike,EUR,819
1,Team Fortress Classic,GBP,429
2,Day of Defeat,GBP,429
3,Deathmatch Classic,GBP,429
4,Half-Life: Opposing Force,GBP,429


In [251]:
##Check if games have a different currency  than gbp
price_data[price_data["currency"] != "GBP"]

,name,currency,price
0,Counter-Strike,EUR,819
24,Portal 2,EUR,975
34,Plants vs. Zombies GOTY Edition,EUR,499
38,Garry's Mod,EUR,999


In [252]:
##Updated processPrice to covert the currency to gbp
def processPrice(df):
    ##Process price_overview column into formatted price column
    df = df.copy()
    
    def parsePrice(x):
        if x is not np.nan:
            return literal_eval(x)
        else:
            return {"currency": "GBP", "initial": -1}
    
    ##Evaluate as dictionary and set to -1 if price missing
    df["price_overview"] = df["price_overview"].apply(parsePrice)
    
    ##Create columns from currency and initial values
    df["currency"] = df["price_overview"].apply(lambda x: x["currency"])
    df["price"] = df["price_overview"].apply(lambda x: x["initial"])
    
    ##Set price of free games to 0
    df.loc[df["is_free"], "price"] = 0
    
    ##Remove non-GBP rows
    df = df[df["currency"] == "GBP"]
    
    ##Change price to display in pounds (only applying to rows with a value greater than 0)
    df.loc[df["price"] > 0, "price"] /= 100
    
    ##Remove columns that are no longer needed
    df = df.drop(["is_free", "currency", "price_overview"], axis = 1)
    
    return df


price_df = processPrice(platforms_df)
price_df[["name", "price"]].head()

C:\Users\lolbo\AppData\Local\Temp\ipykernel_7000\52267678.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 4.29  4.29  4.29  4.29  4.29  7.19  7.19  4.29  8.5   8.5   8.5   8.5
  4.29  8.5   6.69  8.5   5.79  8.5   8.5  14.99  4.29  3.99  7.99  6.99
 14.99  6.99  9.99  4.29  9.99  9.99  9.99  9.99 19.99  4.29  8.5   8.5
  8.5   8.29]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df["price"] > 0, "price"] /= 100


,name,price
1,Team Fortress Classic,4.29
2,Day of Defeat,4.29
3,Deathmatch Classic,4.29
4,Half-Life: Opposing Force,4.29
5,Ricochet,4.29


In [253]:
##Temporarily set a pandas option
with pd.option_context("display.max_colwidth", 500):
    display(price_df[["steam_appid", "packages", "package_groups", "price"]].head())

,steam_appid,packages,package_groups,price
1,20,[29],"[{'name': 'default', 'title': 'Buy Team Fortress Classic', 'description': '', 'selection_text': 'Select a purchase option', 'save_text': '', 'display_type': 0, 'is_recurring_subscription': 'false', 'subs': [{'packageid': 29, 'percent_savings_text': ' ', 'percent_savings': 0, 'option_text': 'Team Fortress Classic - £4.29', 'option_description': '', 'can_get_free_license': '0', 'is_free_license': False, 'price_in_cents_with_discount': 429}]}]",4.29
2,30,"[30, 944613]","[{'name': 'default', 'title': 'Buy Day of Defeat', 'description': '', 'selection_text': 'Select a purchase option', 'save_text': '', 'display_type': 0, 'is_recurring_subscription': 'false', 'subs': [{'packageid': 30, 'percent_savings_text': ' ', 'percent_savings': 0, 'option_text': 'Day of Defeat - £4.29', 'option_description': '', 'can_get_free_license': '0', 'is_free_license': False, 'price_in_cents_with_discount': 429}, {'packageid': 944613, 'percent_savings_text': ' ', 'percent_savings':...",4.29
3,40,[31],"[{'name': 'default', 'title': 'Buy Deathmatch Classic', 'description': '', 'selection_text': 'Select a purchase option', 'save_text': '', 'display_type': 0, 'is_recurring_subscription': 'false', 'subs': [{'packageid': 31, 'percent_savings_text': ' ', 'percent_savings': 0, 'option_text': 'Deathmatch Classic - £4.29', 'option_description': '', 'can_get_free_license': '0', 'is_free_license': False, 'price_in_cents_with_discount': 429}]}]",4.29
4,50,[32],"[{'name': 'default', 'title': 'Buy Half-Life: Opposing Force', 'description': '', 'selection_text': 'Select a purchase option', 'save_text': '', 'display_type': 0, 'is_recurring_subscription': 'false', 'subs': [{'packageid': 32, 'percent_savings_text': ' ', 'percent_savings': 0, 'option_text': 'Half-Life: Opposing Force - £4.29', 'option_description': '', 'can_get_free_license': '0', 'is_free_license': False, 'price_in_cents_with_discount': 429}]}]",4.29
5,60,[33],"[{'name': 'default', 'title': 'Buy Ricochet', 'description': '', 'selection_text': 'Select a purchase option', 'save_text': '', 'display_type': 0, 'is_recurring_subscription': 'false', 'subs': [{'packageid': 33, 'percent_savings_text': ' ', 'percent_savings': 0, 'option_text': 'Ricochet - £4.29', 'option_description': '', 'can_get_free_license': '0', 'is_free_license': False, 'price_in_cents_with_discount': 429}]}]",4.29


In [254]:
##Preview how many rows have price data pf -1
print(price_df[price_df["price"] == -1].shape[0])

2


In [255]:
##Find which rows have no data and how they are stored
print("Null counts:", price_df["package_groups"].isnull().sum())
print("Empty list counts:", price_df[price_df["package_groups"] == "[]"].shape[0])

Null counts: 0
Empty list counts: 4


In [256]:
##Print the rows that have missing price and package group data 
missing_price_and_package = price_df[(price_df["price"] == -1) & (price_df["package_groups"] == "[]")]

print("Number of rows:", missing_price_and_package.shape[0], "\n")

print("First few rows:\n")
printSteamLinks(missing_price_and_package[:5])

print("\nLast few rows:\n")
printSteamLinks(missing_price_and_package[-10:-5])

Number of rows: 2 

First few rows:

Half-Life 2: Lost Coast: https://store.steampowered.com/app/340
Rome: Total War™ - Collection: https://store.steampowered.com/app/4760

Last few rows:



In [257]:
##Print rows that have missing price data but dont have package groups data.
missing_price_have_package = price_df.loc[(price_df["price"] == -1) & (price_df["package_groups"] != "[]"), ["name", "steam_appid", "package_groups", "price"]]

print("Number of rows:", missing_price_have_package.shape[0], "\n")

print("First few rows:\n")
printSteamLinks(missing_price_have_package[:5])

print("\nLast few rows:\n")
printSteamLinks(missing_price_have_package[-10:-5])

Number of rows: 0 

First few rows:


Last few rows:



In [258]:
##Updated processPrice function again to remove the rows that have missing price data but dont have package groups data
def processPrice(df):
    
    df = df.copy()
    
    def parsePrice(x):
        if x is not np.nan:
            return literal_eval(x)
        else:
            return {"currency": "GBP", "initial": -1}
    
    ##Evaluate as dictionary and set to -1 if price missing
    df["price_overview"] = df["price_overview"].apply(parsePrice)
    
    ##Create columns from currency and initial values
    df["currency"] = df["price_overview"].apply(lambda x: x["currency"])
    df["price"] = df["price_overview"].apply(lambda x: x["initial"])
    
    ##Set price of free games to 0
    df.loc[df["is_free"], "price"] = 0
    
    ##Remove thenon-GBP rows
    df = df[df["currency"] == "GBP"]
    
    ##Remove the rows where price is -1
    df = df[df["price"] != -1]
    
    ##Change price to display in pounds (only applying to rows with a value greater than 0)
    df["price"] /= 100
    
    ##Remove columns that are no longer needed
    df = df.drop(["is_free", "currency", "price_overview", "packages", "package_groups"], axis = 1)
    
    return df


price_df = processPrice(platforms_df)
price_df.head()

,name,steam_appid,required_age,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,platforms,metacritic,categories,genres,screenshots,recommendations,release_date,support_info,background,content_descriptors,price
1,Team Fortress Classic,20,0,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,"English, French, German, Italian, Spanish - Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],windows;mac;linux,NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 6000},"{'coming_soon': False, 'date': '1 Apr, 1999'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [2, 5], 'notes': 'Includes intense vio...",4.29
2,Day of Defeat,30,0,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,"English, French, German, Italian, Spanish - Spain",https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],windows;mac;linux,"{'score': 79, 'url': 'https://www.metacritic.c...","[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 4008},"{'coming_soon': False, 'date': '1 May, 2003'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",4.29
3,Deathmatch Classic,40,0,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,"English, French, German, Italian, Spanish - Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Valve'],['Valve'],windows;mac;linux,NaN,"[{'id': 1, 'description': 'Multi-player'}, {'i...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 2104},"{'coming_soon': False, 'date': '1 Jun, 2001'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",4.29
4,Half-Life: Opposing Force,50,0,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,"English, French, German, Korean",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",['Gearbox Software'],['Valve'],windows;mac;linux,NaN,"[{'id': 2, 'description': 'Single-player'}, {'...","[{'id': '1', 'description': 'Action'}]","[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 18753},"{'coming_soon': False, 'date': '1 Nov, 1999'}","{'url': 'https://help.steampowered.com', 'emai...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",4.29
5,Ricochet,60,0,A futuristic action game that challenges your ...,A futuristic action game that challenges your ...,A futuristic action game that challenges your ...,"English, French, German, Italian, Spanish - Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,

In [259]:
##Check for rows with nullvalues
price_df["supported_languages"].isnull().sum()

0

In [260]:
##Preview those rows
price_df[price_df["supported_languages"].isnull()]

,name,steam_appid,required_age,detailed_description,about_the_game,short_description,supported_languages,header_image,website,pc_requirements,mac_requirements,linux_requirements,developers,publishers,platforms,metacritic,categories,genres,screenshots,recommendations,release_date,support_info,background,content_descriptors,price


In [ ]:
##Print the structure of the column
print(price_df["supported_languages"][0])
price_df["supported_languages"].value_counts().head(10)

In [262]:
def processLanguage(df):
    ##Process supported_languages column into a boolean "is english" column
    df = df.copy()
    
    ##Drop rows with missing language data
    df = df.dropna(subset=["supported_languages"])
    
    df["english"] = df["supported_languages"].apply(lambda x: 1 if "english" in x.lower() else 0)
    df = df.drop("supported_languages", axis = 1)
    
    return df


language_df = processLanguage(price_df)
language_df[["name", "english"]].head()

,name,english
1,Team Fortress Classic,1
2,Day of Defeat,1
3,Deathmatch Classic,1
4,Half-Life: Opposing Force,1
5,Ricochet,1


In [263]:
##Display results of the changes
language_df["english"].value_counts()

english
1    43
Name: count, dtype: int64

In [264]:
##Check for null data and empty lists
print("Developers null counts:", language_df["developers"].isnull().sum())
print("Developers empty list counts:", language_df[language_df["developers"] == "[""]"].shape[0])

print("\nPublishers null counts:", language_df["publishers"].isnull().sum())
print("Publishers empty list counts:", language_df[language_df["publishers"] == "[""]"].shape[0])

Developers null counts: 0
Developers empty list counts: 0

Publishers null counts: 0
Publishers empty list counts: 0


In [265]:
##Print rows with missing publishe r abd developer data
no_dev = language_df[language_df["developers"].isnull()]

print("Total games missing developer:", no_dev.shape[0], "\n")

printSteamLinks(no_dev[:5])

no_pub = language_df[language_df["publishers"] == "[""]"]

print("\nTotal games missing publisher:", no_pub.shape[0], "\n")
printSteamLinks(no_pub[:5])

no_dev_or_pub = language_df[(language_df["developers"].isnull()) & (language_df["publishers"] == "[""]")]

print("\nTotal games missing developer and publisher:", no_dev_or_pub.shape[0], "\n")
printSteamLinks(no_dev_or_pub[:5])

Total games missing developer: 0 


Total games missing publisher: 0 


Total games missing developer and publisher: 0 



In [266]:
##Print he structure of the data
language_df[['developers', 'publishers']].iloc[24:28]

,developers,publishers
28,['Valve'],['Valve']
29,['Arkane Studios'],['Ubisoft']
30,['id Software'],['id Software']
31,"['id Software', 'Nightdive Studios', 'MachineG...",['Bethesda Softworks']


In [267]:
def processDevelopersAndPublishers(df):
    ##Remove rows with missing data
    df = df[(df["developers"].notnull()) & (df["publishers"] != "[""]")].copy()
    
    for col in ["developers", "publishers"]:
        df[col] = df[col].apply(lambda x: literal_eval(x))
        
        ##Filter dataframe to rows with lists longer than 1 and store the number of rows
        num_rows = df[df[col].str.len() > 1].shape[0]
        
        print("Rows in {} column with multiple values:".format(col), num_rows)

processDevelopersAndPublishers(language_df)

Rows in developers column with multiple values: 2
Rows in publishers column with multiple values: 4


In [268]:
##Updated processDevelopersAndPublishers so that publishers and developers are joined on the semi colon and create new lists
def processDevelopersAndPublishers(df):
    ##Parses columns as semicolon-separated string
    ##Remove rows with missing data (~ means not)
    df = df[(df["developers"].notnull()) & (df["publishers"] != "['']")].copy()
    df = df[~(df["developers"].str.contains(";")) & ~(df["publishers"].str.contains(";"))]
    df = df[(df["publishers"] != "['NA']") & (df["publishers"] != "['N/A']")]
    
    # create list for each
    df["developer"] = df["developers"].apply(lambda x: ";".join(literal_eval(x)))
    df["publisher"] = df["publishers"].apply(lambda x: ";".join(literal_eval(x)))

    df = df.drop(["developers", "publishers"], axis = 1)
    
    return df

dev_pub_df = processDevelopersAndPublishers(language_df)
dev_pub_df[["name", "steam_appid", "developer", "publisher"]].head()

,name,steam_appid,developer,publisher
1,Team Fortress Classic,20,Valve,Valve
2,Day of Defeat,30,Valve,Valve
3,Deathmatch Classic,40,Valve,Valve
4,Half-Life: Opposing Force,50,Gearbox Software,Valve
5,Ricochet,60,Valve,Valve


In [269]:
def processCategoriesAndGenres(df):
    ##Turns the categoris into a list
    df = df.copy()
    df = df[(df["categories"].notnull()) & (df["genres"].notnull())]
    
    for col in ["categories", "genres"]:
        df[col] = df[col].apply(lambda x: ";".join(item["description"] for item in literal_eval(x)))
    
    return df

cat_gen_df = processCategoriesAndGenres(dev_pub_df)
cat_gen_df[["steam_appid", "categories", "genres"]].head()

,steam_appid,categories,genres
1,20,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action
2,30,Multi-player;Valve Anti-Cheat enabled;Family S...,Action
3,40,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action
4,50,Single-player;Multi-player;Valve Anti-Cheat en...,Action
5,60,Multi-player;PvP;Online PvP;Valve Anti-Cheat e...,Action


In [270]:
def process(df):
    ##Updated process function
    
    ##Make a Copy of the dataframe so you dont have to redownlaod the data each time i make a mistake
    df = df.copy()
    
    ##Remove duplicate rows
    df = df.drop_duplicates()
    
    # Remove collumns with more than 50% null values
    df = dropNullCollumns(df)
    
    # Process rest of columns
    df = processNameType(df)
    df = processAge(df)
    df = processPlatforms(df)
    df = processPrice(df)
    df = processLanguage(df)
    df = processDevelopersAndPublishers(df)
    df = processCategoriesAndGenres(df)
    
    return df

partially_clean = process(raw_steam_data)
partially_clean.head()

,name,steam_appid,required_age,detailed_description,about_the_game,short_description,header_image,website,pc_requirements,mac_requirements,linux_requirements,platforms,metacritic,categories,genres,screenshots,recommendations,release_date,support_info,background,content_descriptors,price,english,developer,publisher
1,Team Fortress Classic,20,0,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",windows;mac;linux,NaN,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 6000},"{'coming_soon': False, 'date': '1 Apr, 1999'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [2, 5], 'notes': 'Includes intense vio...",4.29,1,Valve,Valve
2,Day of Defeat,30,0,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",windows;mac;linux,"{'score': 79, 'url': 'https://www.metacritic.c...",Multi-player;Valve Anti-Cheat enabled;Family S...,Action,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 4008},"{'coming_soon': False, 'date': '1 May, 2003'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",4.29,1,Valve,Valve
3,Deathmatch Classic,40,0,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",windows;mac;linux,NaN,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 2104},"{'coming_soon': False, 'date': '1 Jun, 2001'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",4.29,1,Valve,Valve
4,Half-Life: Opposing Force,50,0,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",windows;mac;linux,NaN,Single-player;Multi-player;Valve Anti-Cheat en...,Action,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 18753},"{'coming_soon': False, 'date': '1 Nov, 1999'}","{'url': 'https://help.steampowered.com', 'emai...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",4.29,1,Gearbox Software,Valve
5,Ricochet,60,0,A futuristic action game that challenges your ...,A futuristic action game that challenges your ...,A futuristic action game that challenges your ...,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",windows;mac;linux,NaN,Multi-player;PvP;Online PvP;Valve Anti-Cheat e...,Action,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 3973},"{'coming_soon': False, 'date': '1 Nov, 2000'}","{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",4.29,1,Valve,Valve


In [271]:
##Checkf or null data in these columns
partially_clean[["detailed_description", "about_the_game", "short_description"]].isnull().sum()

detailed_description    0
about_the_game          0
short_description       0
dtype: int64

In [272]:
def exportData(df, filename):
    ##Export dataframe to csv file, filename prepended with "steam_"

    filepath = "exports/steam_" + filename + ".csv"
    
    df.to_csv(filepath, index=False)
    
    print_name = filename.replace("_", " ")
    print("Exported {} to '{}'".format(print_name, filepath))

In [273]:
def processDescriptions(df, export = False):
    ##Export descriptions to external csv file then remove these columns

    ##Remove rows with missing description data
    df = df[df["detailed_description"].notnull()].copy()
    
    ##Remove rows with unusually small description
    df = df[df["detailed_description"].str.len() > 20]
    
    ##By default we don"t export, useful if calling function later
    if export:
        # create dataframe of description columns
        description_data = df[["steam_appid", "detailed_description", "about_the_game", "short_description"]]
        
        exportData(description_data, filename="description_data")
    
    ##Drop description columns from main dataframe
    df = df.drop(["detailed_description", "about_the_game", "short_description"], axis=1)
    
    return df

desc_df = processDescriptions(partially_clean, export=True)

Exported description data to 'exports/steam_description_data.csv'


In [274]:
#Inspect exported data
pd.read_csv("exports/steam_description_data.csv").head()

,steam_appid,detailed_description,about_the_game,short_description
0,20,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...
1,30,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...
2,40,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...
3,50,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...
4,60,A futuristic action game that challenges your ...,A futuristic action game that challenges your ...,A futuristic action game that challenges your ...


In [275]:
##Check for misssing values in the images columns
image_cols = ['header_image', 'screenshots', 'background']

for col in image_cols:
    print(col+':', desc_df[col].isnull().sum())

desc_df[image_cols].head()

header_image: 0
screenshots: 0
background: 0


,header_image,screenshots,background
1,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...
2,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...
3,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...
4,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...
5,https://cdn.akamai.steamstatic.com/steam/apps/...,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",https://cdn.akamai.steamstatic.com/steam/apps/...


In [276]:
no_screenshots = desc_df[desc_df["screenshots"].isnull()]

printSteamLinks(no_screenshots)

no_screenshots

,name,steam_appid,required_age,header_image,website,pc_requirements,mac_requirements,linux_requirements,platforms,metacritic,categories,genres,screenshots,recommendations,release_date,support_info,background,content_descriptors,price,english,developer,publisher


In [277]:
def processReleaseDate(df):
    df = df.copy()
    
    def eval_date(x):
        x = literal_eval(x)
        if x["coming_soon"]:
            return "" ##Return blank string so can drop missing at end
        else:
            return x["date"]
    
    df["release_date"] = df["release_date"].apply(eval_date)
    
    def parse_date(x):
        if re.search(r"[\d]{1,2} [A-Za-z]{3}, [\d]{4}", x):
            return x.replace(",", "")
        elif re.search(r"[A-Za-z]{3} [\d]{4}", x):
            return "1 " + x
        elif x == "":
            return np.nan
        else:
            ##Should be everything, print out anything left just in case
            print(x)
            
    df["release_date"] = df["release_date"].apply(parse_date)
    df["release_date"] = pd.to_datetime(df["release_date"], format = "%d %b %Y", errors = "coerce")
    
    df = df[df["release_date"].notnull()]
    
    return df

In [278]:
def process(df):
    ##Final Process function has all the cleaning functions added
    df
    
    ##Make a Copy of the dataframe so you dont have to redownlaod the data each time i make a mistake
    df = df.copy()
    
    ##Remove duplicate rows
    df = df.drop_duplicates()
    
    ##Remove collumns with more than 50% null values
    df = dropNullCollumns(df)

    ##Drop all the unused Columns
    
    ##Process rest of columns
    df = processNameType(df)
    df = processAge(df)
    df = processPlatforms(df)
    df = processPrice(df)
    df = processLanguage(df)
    df = processDevelopersAndPublishers(df)
    df = processCategoriesAndGenres(df) 
    df = processReleaseDate(df)
    
    ##Process columns which export data
    df = processDescriptions(df, export=True)
    return df

steam_data = process(raw_steam_data)
steam_data.head()

Exported description data to 'exports/steam_description_data.csv'


,name,steam_appid,required_age,header_image,website,pc_requirements,mac_requirements,linux_requirements,platforms,metacritic,categories,genres,screenshots,recommendations,release_date,support_info,background,content_descriptors,price,english,developer,publisher
1,Team Fortress Classic,20,0,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",windows;mac;linux,NaN,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 6000},1999-04-01,"{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [2, 5], 'notes': 'Includes intense vio...",4.29,1,Valve,Valve
2,Day of Defeat,30,0,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.dayofdefeat.com/,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",windows;mac;linux,"{'score': 79, 'url': 'https://www.metacritic.c...",Multi-player;Valve Anti-Cheat enabled;Family S...,Action,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 4008},2003-05-01,"{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",4.29,1,Valve,Valve
3,Deathmatch Classic,40,0,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",windows;mac;linux,NaN,Multi-player;PvP;Online PvP;Shared/Split Scree...,Action,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 2104},2001-06-01,"{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",4.29,1,Valve,Valve
4,Half-Life: Opposing Force,50,0,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",windows;mac;linux,NaN,Single-player;Multi-player;Valve Anti-Cheat en...,Action,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 18753},1999-11-01,"{'url': 'https://help.steampowered.com', 'emai...",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",4.29,1,Gearbox Software,Valve
5,Ricochet,60,0,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...",windows;mac;linux,NaN,Multi-player;PvP;Online PvP;Valve Anti-Cheat e...,Action,"[{'id': 0, 'path_thumbnail': 'https://cdn.akam...",{'total': 3973},2000-11-01,"{'url': '', 'email': ''}",https://cdn.akamai.steamstatic.com/steam/apps/...,"{'ids': [], 'notes': None}",4.29,1,Valve,Valve


In [279]:
##Final check for any null data
steam_data.isnull().sum()

name                    0
steam_appid             0
required_age            0
header_image            0
website                12
pc_requirements         0
mac_requirements        0
linux_requirements      0
platforms               0
metacritic             11
categories              0
genres                  0
screenshots             0
recommendations         0
release_date            0
support_info            0
background              0
content_descriptors     0
price                   0
english                 0
developer               0
publisher               0
dtype: int64

In [280]:
##this checks for any unreleased games, (the data may have to change depending on when it was run)
steam_data[steam_data["release_date"] > "2024-02-13"]

,name,steam_appid,required_age,header_image,website,pc_requirements,mac_requirements,linux_requirements,platforms,metacritic,categories,genres,screenshots,recommendations,release_date,support_info,background,content_descriptors,price,english,developer,publisher


In [281]:
##Final export of clean steam data 
steam_data.to_csv("exports/steam_data_clean.csv", index = False)